### Basic Setup
Here we check the sage version, configure some settings, define a 2-manifold $M$ with cylidrical coordinate chart $Y$, and a riemannian metric. The naming convention python variables (not variables used in the actual expressions) is `CamelCase` for objects defined on the manifold, and `snake_case` (with a trailing underscore for functions) for generic symbolic expressions. For rank $(0, 2)$ tensors, we default to the purely covariant form (as that is what we have regularization conditions for), denote the $(1, 1)$ version as `X_mat` and denote the rank $(2, 0)$ version as `X_con`.

In [54]:
# Check version
version()

'SageMath version 9.5, Release Date: 2022-01-30'

In [55]:
# Reset
reset()
# Setup pretty printing
%display latex

In [56]:
# Define a differentiable manifold of dimension 2 over real numbers
M = Manifold(2, 'M', latex_name=r'\mathcal{M}', start_index=1)
# Define a cylindrical chart on the manifold
Chart.<rho, z> = M.chart(r'rho:\rho z:z')
# Cache frame
Chartf = Chart.frame()

In [57]:
# Declare a metric on the manifold
G = M.riemannian_metric('g')

# Components of Metric
grr_ = function('grr_f', latex_name='g_{rr}')(rho, z)
gzz_ = function('gzz_f', latex_name='g_{zz}')(rho, z)
grz_ = function('grz_f', latex_name='g_{rz}')(rho, z)

# Metric is symmetric
G[1, 1], G[2, 2] = grr_, gzz_
G[1, 2] = grz_

# Invert metric
G_con = G.inverse()

G_det = M.scalar_field(grr_ * gzz_ - grz_ * grz_, chart=Chart, name='g_det', latex_name=r'\det{g}')

# Print metric
G.display()

g = grr_f(rho, z) drho⊗drho + grz_f(rho, z) drho⊗dz + grz_f(rho, z) dz⊗drho + gzz_f(rho, z) dz⊗dz

In [58]:
G_con.display()

inv_g = -gzz_f(rho, z)/(grz_f(rho, z)^2 - grr_f(rho, z)*gzz_f(rho, z)) ∂/∂rho⊗∂/∂rho + grz_f(rho, z)/(grz_f(rho, z)^2 - grr_f(rho, z)*gzz_f(rho, z)) ∂/∂rho⊗∂/∂z + grz_f(rho, z)/(grz_f(rho, z)^2 - grr_f(rho, z)*gzz_f(rho, z)) ∂/∂z⊗∂/∂rho - grr_f(rho, z)/(grz_f(rho, z)^2 - grr_f(rho, z)*gzz_f(rho, z)) ∂/∂z⊗∂/∂z

In [59]:
# Compute Ricci Tensor
R = G.ricci()
# And Ricci Scalar
R_trace = G_con['^ij'] * R['_ij']
# And connection coefficients, used for covariant derivative
Nabla = G.connection(name='nabla', latex_name=r'\nabla')
# Make sure metric is compatible
assert(Nabla(G) == 0)

Nabla.display()

Gam^rho_rho,rho = -1/2*(gzz_f(rho, z)*d(grr_f)/drho + grz_f(rho, z)*d(grr_f)/dz - 2*grz_f(rho, z)*d(grz_f)/drho)/(grz_f(rho, z)^2 - grr_f(rho, z)*gzz_f(rho, z)) 
Gam^rho_rho,z = -1/2*(gzz_f(rho, z)*d(grr_f)/dz - grz_f(rho, z)*d(gzz_f)/drho)/(grz_f(rho, z)^2 - grr_f(rho, z)*gzz_f(rho, z)) 
Gam^rho_z,rho = -1/2*(gzz_f(rho, z)*d(grr_f)/dz - grz_f(rho, z)*d(gzz_f)/drho)/(grz_f(rho, z)^2 - grr_f(rho, z)*gzz_f(rho, z)) 
Gam^rho_z,z = -1/2*(2*gzz_f(rho, z)*d(grz_f)/dz - gzz_f(rho, z)*d(gzz_f)/drho - grz_f(rho, z)*d(gzz_f)/dz)/(grz_f(rho, z)^2 - grr_f(rho, z)*gzz_f(rho, z)) 
Gam^z_rho,rho = 1/2*(grz_f(rho, z)*d(grr_f)/drho + grr_f(rho, z)*d(grr_f)/dz - 2*grr_f(rho, z)*d(grz_f)/drho)/(grz_f(rho, z)^2 - grr_f(rho, z)*gzz_f(rho, z)) 
Gam^z_rho,z = 1/2*(grz_f(rho, z)*d(grr_f)/dz - grr_f(rho, z)*d(gzz_f)/drho)/(grz_f(rho, z)^2 - grr_f(rho, z)*gzz_f(rho, z)) 
Gam^z_z,rho = 1/2*(grz_f(rho, z)*d(grr_f)/dz - grr_f(rho, z)*d(gzz_f)/drho)/(grz_f(rho, z)^2 - grr_f(rho, z)*gzz_f(rho, z)) 
Gam^z_z,z = 1/2*(2*grz_f(rho, z)*d(grz_f)/dz - grz_f(rho, z)*d(gzz_f)/drho - grr_f(rho, z)*d(gzz_f)/dz)/(grz_f(rho, z)^2 - grr_f(rho, z)*gzz_f(rho, z))

### Variables
Next we define the relavent tensor fields, as well as their components in our default frame.

In [60]:
# Seed Function
s_ = function('s_f', latex_name=r's')(rho, z)

# Resulting Tensors
Lam = M.scalar_field(rho * exp(rho * s_) * sqrt(grr_), chart=Chart, name='lam_s', latex_name=r'\lambda')
          
Lam.display()

lam_s: M → ℝ
   (rho, z) ↦ rho*e^(rho*s_f(rho, z))*sqrt(grr_f(rho, z))

In [61]:
# Gauge Fields
lapse_ = function('lapse_f', latex_name=r'\alpha')(rho, z)

shiftr_ = function('shiftr_f', latex_name=r'\beta^r')(rho, z)
shiftz_ = function('shiftz_f', latex_name=r'\beta^z')(rho, z)

# Gauge Tensors
Lapse = M.scalar_field(lapse_, chart=Chart, name='Lapse', latex_name=r'\alpha')

Shift = M.tensor_field(1, 0, name='Shift', latex_name=r'\beta')
Shift.add_comp(Chartf)[1] = shiftr_
Shift.add_comp(Chartf)[2] = shiftz_

Lapse.display()

Lapse: M → ℝ
   (rho, z) ↦ lapse_f(rho, z)

In [62]:
Shift.display()

Shift = shiftr_f(rho, z) ∂/∂rho + shiftz_f(rho, z) ∂/∂z

In [63]:
# Extrinsic Curvature

# Components
krr_ = function('Krr_f', latex_name='K_{rr}')(rho, z)
kzz_ = function('Kzz_f', latex_name='K_{zz}')(rho, z)
krz_ = function('Krz_f', latex_name='K_{rz}')(rho, z)

# Phi Component of K
y_ = function('y_f', latex_name=r'Y')(rho, z)

K = M.tensor_field(0, 2, sym=(0, 1), name='K')
K.add_comp(Chartf)[1, 1] = krr_
K.add_comp(Chartf)[2, 2] = kzz_
K.add_comp(Chartf)[1, 2] = krz_

K_mat = G_con['^{ij}'] * K['_jk']
K_con = K_mat['^i_k'] * G_con['^{kj}']
K_trace = G_con['^ij'] * K['_ij']

# L = K_\phi^\phi
L = M.scalar_field(krr_/grr_ + rho * y_, name='L')

K.display()

K = Krr_f(rho, z) drho⊗drho + Krz_f(rho, z) drho⊗dz + Krz_f(rho, z) dz⊗drho + Kzz_f(rho, z) dz⊗dz

In [64]:
L.display()

L: M → ℝ
   (rho, z) ↦ rho*y_f(rho, z) + Krr_f(rho, z)/grr_f(rho, z)

In [65]:
# Z4
theta_ = function('theta_f', latex_name=r'\theta')(rho, z)
Theta = M.scalar_field(theta_, chart=Chart, name='theta_s', latex_name=r'\theta')

zr_ = function('Zr_f', latex_name=r'Z_r')(rho, z)
zz_ = function('Zz_f', latex_name=r'Z_z')(rho, z)

Z = M.tensor_field(0, 1, name='Z')
Z.add_comp(Chartf)[1] = zr_
Z.add_comp(Chartf)[2] = zz_

# Vector form of Z
Zv = G_con['^ij'] * Z['_j']

Theta.display()

theta_s: M → ℝ
   (rho, z) ↦ theta_f(rho, z)

In [66]:
Z.display()

Z = Zr_f(rho, z) drho + Zz_f(rho, z) dz

### Constraint equations
Constraint equations that should be satisfied at all times.
$$ \mathcal{C} \equiv \frac{1}{2} (K^2 - K_{ij} K^{ij} + R) - \lambda^{-1} \nabla^j \nabla_j \lambda  + K L = 0 $$

$$ \mathcal{C}_i \equiv \nabla_j K_{i}^{\;j} - \nabla_i (K + L) + \lambda^{-1} (\nabla_j \lambda) K_{i}^{\;j} - \lambda^{-1} (\nabla_i \lambda) L = 0$$

In [67]:
term1 = (K_trace^2 - K['_ij'] * K_con['^ij'] + R_trace) / 2
term2 = - (Nabla(Nabla(Lam)) / Lam)['_ij'] * G_con['^{ij}'] 
term3 = K_trace * L

# Hamiltonian
CH = term1 + term2 + term3

CH

Scalar field on the 2-dimensional differentiable manifold M

In [68]:
# term1 = Nabla(K)['_ijk'] * G_con['^jk'] - Nabla(K_trace + L)
# term2 = LamLog['_j'] * K_mat['^j_i'] - LamLog * L

term1 = Nabla(K)['_ijk'] * G_con['^jk'] - Nabla(K_trace + L)
term2 = (Nabla(Lam) / Lam)['_i'] * K_mat['^i_j']
term3 = -(Nabla(Lam) / Lam) * L

# Momentum Constraint
CM = term1 + term2 + term3

CM

1-form on the 2-dimensional differentiable manifold M

### Evolution Equations
Calculations for $\mathcal{L}_n X$ for various dynamical quantities.

In [69]:
term1 = R - Nabla(Nabla(Lam)) / Lam - Nabla(Nabla(Lapse)) / Lapse
term2 = (K_trace + L) * K - 2 * K['_ij'] * K_mat['^j_k']
term3 = 2 * Nabla(Z)['_(ij)'] - 2 * K * Theta

# Extrinsic Curvature
LieK = term1 + term2 + term3

In [70]:
term1 = -(Nabla(Nabla(Lam)) / Lam)['_ij'] * G_con['^ij']
term2 = -(Nabla(Lam) / Lam * Nabla(Lapse) / Lapse)['_ij'] * G_con['^ij']
term3 = L * (K_trace + L)
term4 = 2 * (Nabla(Lam) / Lam)['_i'] * Zv['^i'] - 2 * L * Theta

# Angular Extrinsic Curvature
LieL = term1 + term2 + term3 + term4

In [71]:
# Metric and Lambda
LieG = -2*K
LieLam = -Lam * L

In [72]:
term1 = CH + (Nabla(Lam) / Lam - Nabla(Lapse) / Lapse)['_i'] * Zv['^i']
term2 = Nabla(Zv)['^i_i'] - (K_trace + L) * Theta

# Theta
LieTheta = term1 + term2

In [73]:
term1 = CM - 2 * K['_ij'] * Zv['^j']
term2 = - Nabla(Lapse) / Lapse * Theta + Nabla(Theta) 

# Z
LieZ = term1 + term2

In [74]:
# Actual temporal derivatives including shift terms
G_t = Lapse * LieG + G.lie_derivative(Shift)
Lam_t = Lapse * LieLam + Lam.lie_derivative(Shift)
K_t = Lapse * LieK + K.lie_derivative(Shift)
L_t = Lapse * LieL + L.lie_derivative(Shift)

Theta_t = Lapse * LieTheta + Theta.lie_derivative(Shift)
Z_t = Lapse * LieZ + Z.lie_derivative(Shift)

Y_t = (L_t - K_t[1, 1] / G[1, 1] + K[1, 1] / G[1, 1]^2 * G_t[1, 1]) / rho
s_t = (Lam_t / Lam - G_t[1, 1] / (2 * G[1, 1])) / rho

### Gauge Evolution

Here we derive equations for the gauge evolution. Because these equations involve partial derivatives and not covariant ones (harmonic gauge conditions), all results are expressions rather than tensors).

In [75]:
# Constants
f_c = 1
mu_c = 1
d_c = 1
a_c = 1
m_c = 2

In [76]:
term1 = (- Lapse^2 * f_c * (K_trace + L - m_c * Theta)).expr()
term2 = shiftr_ * diff(lapse_, rho) + shiftz_ * diff(lapse_, z)

lapse_t_ = (term1 + term2)

In [77]:
LogLamG = (Nabla(Lam) / Lam + 1/2 * Nabla(G_det) / G_det)['_i'] * G_con['^ij']
LogLamMuD = 2 * mu_c * (LogLamG - Zv) - d_c * LogLamG + a_c * (Nabla(Lapse) / Lapse)['_i'] * G_con['^ij']


harmonicr_ = diff(G_con[1, 1].expr(), rho) + diff(G_con[1, 2].expr(), z)
harmonicz_ = diff(G_con[2, 1].expr(), rho) + diff(G_con[2, 2].expr(), z)

shiftadvr_ = shiftr_ * diff(shiftr_, rho) + shiftz_ * diff(shiftr_, z)
shiftadvz_ = shiftr_ * diff(shiftz_, rho) + shiftz_ * diff(shiftz_, z)

shiftr_t_ = -lapse_^2 * (LogLamMuD[1].expr() + mu_c * harmonicr_ - (2 * mu_c - d_c) * G_con[1, 1].expr() / rho) + shiftadvr_
shiftz_t_ = -lapse_^2 * (LogLamMuD[2].expr() + mu_c * harmonicz_) + shiftadvz_

### Preprocessing
To avoid transcription errors when transferring between notebooks and code we automatically transform symbolic expressions into `C` code.

In [100]:
# Symbolic Variables to replace current functions
var('grr_r grr_z grr_rr grr_zz grr_rz grr')
var('gzz_r gzz_z gzz_rr gzz_zz gzz_rz gzz')
var('grz_r grz_z grz_rr grz_zz grz_rz grz')
var('s_r s_z s_rr s_zz s_rz s')

var('krr_r krr_z krr')
var('kzz_r kzz_z kzz')
var('krz_r krz_z krz')
var('y_r y_z y')

var('lapse_r lapse_z lapse_rr lapse_zz lapse_rz lapse')
var('shiftr_r shiftr_z shiftz_r shiftz_z shiftr shiftz')

var('theta_r theta_z theta')
var('zr_r zr_z zz_r zz_z zr zz')

"""
Preprocesses an expression, replacing all derivatives of a function
and invokations of that functions with an appropriately named variable.
"""
def process(expr):
    expr = expr.expand()
    expr = expr.subs({
        diff(grr_, rho): grr_r,
        diff(grr_, z): grr_z,
        diff(grr_, rho, rho): grr_rr,
        diff(grr_, z, z): grr_zz,
        diff(grr_, rho, z): grr_rz,
        diff(grr_, z, rho): grr_rz,
        
        diff(gzz_, rho): gzz_r,
        diff(gzz_, z): gzz_z,
        diff(gzz_, rho, rho): gzz_rr,
        diff(gzz_, z, z): gzz_zz,
        diff(gzz_, rho, z): gzz_rz,
        diff(gzz_, z, rho): gzz_rz,
        
        diff(grz_, rho): grz_r,
        diff(grz_, z): grz_z,
        diff(grz_, rho, rho): grz_rr,
        diff(grz_, z, z): grz_zz,
        diff(grz_, rho, z): grz_rz,
        diff(grz_, z, rho): grz_rz,
        
        diff(krr_, rho): krr_r,
        diff(krr_, z): krr_z,
        
        diff(kzz_, rho): kzz_r,
        diff(kzz_, z): kzz_z,
        
        diff(krz_, rho): krz_r,
        diff(krz_, z): krz_z,
        
        diff(s_, rho): s_r,
        diff(s_, z): s_z,
        diff(s_, rho, rho): s_rr,
        diff(s_, z, z): s_zz,
        diff(s_, rho, z): s_rz,
        diff(s_, z, rho): s_rz,
        
        diff(lapse_, rho): lapse_r,
        diff(lapse_, z): lapse_z,
        diff(lapse_, rho, rho): lapse_rr,
        diff(lapse_, z, z): lapse_zz,
        diff(lapse_, rho, z): lapse_rz,
        diff(lapse_, z, rho): lapse_rz,
        
        diff(shiftr_, rho): shiftr_r,
        diff(shiftr_, z): shiftr_z,
        diff(shiftz_, rho): shiftz_r,
        diff(shiftz_, z): shiftz_z,
        
        diff(y_, rho): y_r,
        diff(y_, z): y_z,
        
        diff(theta_, rho): theta_r,
        diff(theta_, z): theta_z,
        
        diff(zr_, rho): zr_r,
        diff(zr_, z): zr_z,
        diff(zz_, rho): zz_r,
        diff(zz_, z): zz_z,
    })
    expr = expr.subs({
        grr_: grr,
        gzz_: gzz,
        grz_: grz,
        s_: s,
        
        krr_: krr,
        kzz_: kzz,
        krz_: krz,
        y_: y,

        lapse_: lapse,
        shiftr_: shiftr,
        shiftz_: shiftz,
        
        theta_: theta,
        zr_: zr,
        zz_: zz,
    })
    return expr

# def process(expr):
#     expr = expr._sympy_().expand()
#     expr = expr.expand().subs([
#         (diff(grr_, rho), grr_r),
#         (diff(grr_, z), grr_z),
#         (diff(grr_, rho, rho), grr_rr),
#         (diff(grr_, z, z), grr_zz),
#         (diff(grr_, rho, z), grr_rz),
#         (diff(grr_, z, rho), grr_rz),
        
#         (diff(gzz_, rho), gzz_r),
#         (diff(gzz_, z), gzz_z),
#         (diff(gzz_, rho, rho), gzz_rr),
#         (diff(gzz_, z, z), gzz_zz),
#         (diff(gzz_, rho, z), gzz_rz),
#         (diff(gzz_, z, rho), gzz_rz),
        
#         (diff(grz_, rho), grz_r),
#         (diff(grz_, z), grz_z),
#         (diff(grz_, rho, rho), grz_rr),
#         (diff(grz_, z, z), grz_zz),
#         (diff(grz_, rho, z), grz_rz),
#         (diff(grz_, z, rho), grz_rz),
        
#         (diff(krr_, rho), krr_r),
#         (diff(krr_, z), krr_z),
        
#         (diff(kzz_, rho), kzz_r),
#         (diff(kzz_, z), kzz_z),
        
#         (diff(krz_, rho), krz_r),
#         (diff(krz_, z), krz_z),
        
#         (diff(s_, rho), s_r),
#         (diff(s_, z), s_z),
#         (diff(s_, rho, rho), s_rr),
#         (diff(s_, z, z), s_zz),
#         (diff(s_, rho, z), s_rz),
#         (diff(s_, z, rho), s_rz),
        
#         (diff(lapse_, rho), lapse_r),
#         (diff(lapse_, z), lapse_z),
#         (diff(lapse_, rho, rho), lapse_rr),
#         (diff(lapse_, z, z), lapse_zz),
#         (diff(lapse_, rho, z), lapse_rz),
#         (diff(lapse_, z, rho), lapse_rz),
        
#         (diff(shiftr_, rho), shiftr_r),
#         (diff(shiftr_, z), shiftr_z),
#         (diff(shiftz_, rho), shiftz_r),
#         (diff(shiftz_, z), shiftz_z),
        
#         (diff(y_, rho), y_r),
#         (diff(y_, z), y_z),
        
#         (diff(theta_, rho), theta_r),
#         (diff(theta_, z), theta_z),
        
#         (diff(zr_, rho), zr_r),
#         (diff(zr_, z), zr_z),
#         (diff(zz_, rho), zz_r),
#         (diff(zz_, z), zz_z),
#     ])
#     expr = expr.expand().subs([
#         (grr_, grr),
#         (gzz_, gzz),
#         (grz_, grz),
#         (s_, s),
        
#         (krr_, krr),
#         (kzz_, kzz),
#         (krz_, krz),
#         (y_, y),

#         (lapse_, lapse),
#         (shiftr_, shiftr),
#         (shiftz_, shiftz),
        
#         (theta_, theta),
#         (zr_, zr),
#         (zz_, zz),
#     ])
    
#     return expr._sage_()

In [101]:
def regularize_diagonal(expr):
    expr = expr._sympy_().expand()
    expr = expr.expand().subs(grz / rho, grz_r)
    expr = expr.expand().subs(grz_z / rho, grz_rz)
    expr = expr.expand().subs(grz, 0)
    expr = expr.expand().subs(grz_z, 0)
    return expr._sage_()

def regularize_metric_values(expr):
    expr = expr._sympy_().expand()
    expr = expr.expand().subs(grz, 0)
    expr = expr.expand().subs(grz_z, 0)
    expr = expr.expand().subs(grz_zz, 0)
    expr = expr.expand().subs(grr_r, 0)
    expr = expr.expand().subs(gzz_r, 0)
    expr = expr.expand().subs(grr_rz, 0)
    expr = expr.expand().subs(gzz_rz, 0)
    return expr._sage_()

def regularize_metric_derivs(expr):
    expr = expr._sympy_().expand()
    expr = expr.expand().subs(grz / rho, grz_r)
    expr = expr.expand().subs(grz_z / rho, grz_rz)
    expr = expr.expand().subs(grr_r / rho, grr_rr)
    expr = expr.expand().subs(gzz_r / rho, gzz_rr)
    return expr._sage_()

def regularize_extrinsic_values(expr):
    expr = expr._sympy_().expand()
    expr = expr.expand().subs(krz, 0)
    expr = expr.expand().subs(krz_z, 0)
    expr = expr.expand().subs(krr_r, 0)
    expr = expr.expand().subs(kzz_r, 0)
    return expr._sage_()

def regularize_extrinsic_derivs(expr):
    expr = expr._sympy_().expand()
    expr = expr.expand().subs(krz / rho, krz_r)
    return expr._sage_()


def regularize_gauge_values(expr):
    expr = expr._sympy_().expand()
    expr = expr.expand().subs(lapse_r, 0)
    expr = expr.expand().subs(lapse_rz, 0)
    expr = expr.expand().subs(shiftr, 0)
    expr = expr.expand().subs(shiftr_z, 0)
    expr = expr.expand().subs(shiftz_r, 0)
    return expr._sage_()

def regularize_gauge_derivs(expr):
    expr = expr._sympy_().expand()
    expr = expr.expand().subs(lapse_r / rho, lapse_rr)
    expr = expr.expand().subs(shiftr / rho, shiftr_r)
    return expr._sage_()

def regularize_constraint_values(expr):
    expr = expr._sympy_().expand()
    expr = expr.expand().subs(theta_r, 0)
    expr = expr.expand().subs(zr, 0)
    expr = expr.expand().subs(zr_z, 0)
    expr = expr.expand().subs(zz_r, 0)
    return expr._sage_()

def regularize_constraint_derivs(expr):
    expr = expr._sympy_().expand()
    expr = expr.expand().subs(zr / rho, zr_r)
    return expr._sage_()

def regularize_angular_values(expr):
    expr = expr._sympy_().expand()
    expr = expr.expand().subs(s, 0)
    expr = expr.expand().subs(s_z, 0)
    expr = expr.expand().subs(s_zz, 0)
    expr = expr.expand().subs(y, 0)
    expr = expr.expand().subs(y_z, 0)
    return expr._sage_()

def regularize_angular_derivs(expr):
    expr = expr._sympy_().expand()
    expr = expr.expand().subs(s / rho, s_r)
    expr = expr.expand().subs(s_z / rho, s_rz)
    expr = expr.expand().subs(y / rho, y_r)
    return expr._sage_()

def regularize(expr):
    expr = process(expr)
    # This could be destructive
    expr = regularize_diagonal(expr)
    # This shouldn't be
    expr = regularize_metric_derivs(expr)
    expr = regularize_extrinsic_derivs(expr)
    expr = regularize_gauge_derivs(expr)
    expr = regularize_constraint_derivs(expr)
    expr = regularize_angular_derivs(expr)
    # And anything after this is checked by NaNs
    expr = expr._sympy_().expand().subs(rho, 0)._sage_()
    
    expr = regularize_metric_values(expr)
    expr = regularize_extrinsic_values(expr)
    expr = regularize_gauge_values(expr)
    expr = regularize_constraint_values(expr)
    expr = regularize_angular_values(expr)
    
    return expr

In [102]:
import sympy as sym

def generate_ccode(eqs, func):
    names = []
    exprs = []
    
    for (name, eq) in eqs:
        names.append(name)
        exprs.append(func(eq)._sympy_().simplify())
        
    subs, final = sym.cse(exprs)
    
    result = ("/*************************************\n" + 
              "This code was generated automatically\n" +
              "using Sagemath and SymPy\n" +
              "**************************************/\n")
    result += "\n// Subexpressions\n"
    
    for (name, expr) in subs:
        code = sym.ccode(expr)
        result += "double " + str(name) + " = " + str(code) + ";\n"
        
    result += "\n// Final Equations\n"
        
    for (name, expr) in zip(names, final):
        code = sym.ccode(expr)
        result += "double " + str(name) + " = " + str(code) + ";\n\n"
        
    return result

In [103]:
hyperbolic = [
    ("grr_t", G_t[1, 1].expr()),
    ("gzz_t", G_t[2, 2].expr()),
    ("grz_t", G_t[1, 2].expr()),
    ("krr_t", K_t[1, 1].expr()),
    ("kzz_t", K_t[2, 2].expr()),
    ("krz_t", K_t[1, 2].expr()),
    ("theta_t", Theta_t.expr()),
    ("zr_t", Z_t[1].expr()),
    ("zz_t", Z_t[2].expr()),
    ("y_t", Y_t.expr()),
    ("s_t", s_t.expr()),
    ("lapse_t", lapse_t_),
    ("shiftr_t", shiftr_t_),
    ("shiftz_t", shiftz_t_),
]

code = generate_ccode(hyperbolic, process)

hyperbolic_h = open("hyperbolic.h", 'w')
hyperbolic_h.write(code)
hyperbolic_h.close()

print(code)

/*************************************
This code was generated automatically
using Sagemath and SymPy
**************************************/

// Subexpressions
double x0 = grr*shiftr_r;
double x1 = 2*x0;
double x2 = 2*grz;
double x3 = shiftz_r*x2;
double x4 = 2*lapse;
double x5 = krr*x4;
double x6 = 2*gzz;
double x7 = shiftz_z*x6;
double x8 = kzz*x4;
double x9 = grr*shiftr_z;
double x10 = gzz*shiftz_r;
double x11 = krz*x4;
double x12 = pow(grr, 4);
double x13 = gzz_z*x12;
double x14 = pow(grr, 2);
double x15 = pow(grr_z, 2);
double x16 = pow(grz, 2);
double x17 = x15*x16;
double x18 = pow(grr, 3);
double x19 = gzz_z*x18;
double x20 = grz*x19;
double x21 = gzz*x18;
double x22 = gzz_r*x21;
double x23 = grz*gzz_r;
double x24 = x18*x23;
double x25 = pow(grr_r, 2);
double x26 = grr*gzz;
double x27 = x16*x26;
double x28 = 3*x27;
double x29 = pow(gzz_r, 2);
double x30 = pow(grz, 4);
double x31 = x12*x29 + x25*x30;
double x32 = lapse*rho;
double x33 = (1.0/4.0)*x32;
double x34 = pow(grz, 3);


In [104]:
expr = process(Y_t.expr())
# This could be destructive
# expr = regularize_diagonal(expr)
# This shouldn't be
# expr = regularize_metric_derivs(expr)
# expr = regularize_extrinsic_derivs(expr)
# expr = regularize_gauge_derivs(expr)
# expr = regularize_constraint_derivs(expr)
# expr = regularize_angular_derivs(expr)

expr = regularize_diagonal(expr)

# expr = regularize_metric_derivs(expr)
# expr = regularize_extrinsic_derivs(expr)
# expr = regularize_gauge_derivs(expr)
# expr = regularize_constraint_derivs(expr)
# expr = regularize_angular_derivs(expr)


expr

lapse*rho*y^2 - lapse*rho*s_z^2/gzz - 2*lapse*theta*y + 2*krr*lapse*y/grr + kzz*lapse*y/gzz + shiftr*y_r + shiftz*y_z + 2*lapse*s_r*zr/grr + 2*lapse*s_z*zz/gzz - 1/2*gzz_r*lapse*s_r/(grr*gzz) - lapse_r*s_r/grr - grr_z*lapse*s_z/(grr*gzz) + 1/2*gzz_z*lapse*s_z/gzz^2 - lapse_z*s_z/gzz - lapse*s_zz/gzz + shiftr*y/rho + 2*lapse*s*zr/(grr*rho) + 2*krz^2*lapse/(grr*gzz*rho) - 1/2*gzz_r*lapse*s/(grr*gzz*rho) - lapse_r*s/(grr*rho) - 2*krz*shiftz_r/(grr*rho) + 2*grr_r*lapse*zr/(grr^2*rho) - 2*lapse*zr_r/(grr*rho) + 2*grz_r*lapse*zz/(grr*gzz*rho) - grz_rz*lapse/(grr*gzz*rho) - 1/2*grr_r*gzz_r*lapse/(grr^2*gzz*rho) - 1/4*gzz_r^2*lapse/(grr*gzz^2*rho) + 1/2*gzz_rr*lapse/(grr*gzz*rho) + 1/2*grz_r*gzz_z*lapse/(grr*gzz^2*rho) - grr_r*lapse_r/(grr^2*rho) + lapse_rr/(grr*rho) - grz_r*lapse_z/(grr*gzz*rho) + 2*lapse*zr/(grr*rho^2) - 1/2*gzz_r*lapse/(grr*gzz*rho^2) - lapse_r/(grr*rho^2)

In [105]:
hyperbolic_regular = [
    ("grr_t", G_t[1, 1].expr()),
    ("gzz_t", G_t[2, 2].expr()),
    ("grz_t", G_t[1, 2].expr()),
    ("krr_t", K_t[1, 1].expr()),
    ("kzz_t", K_t[2, 2].expr()),
    ("krz_t", K_t[1, 2].expr()),
    ("theta_t", Theta_t.expr()),
    ("zr_t", Z_t[1].expr()),
    ("zz_t", Z_t[2].expr()),
    ("y_t", SR(0)),
    ("s_t", s_t.expr()),
    ("lapse_t", lapse_t_),
    ("shiftr_t", shiftr_t_),
    ("shiftz_t", shiftz_t_),
]

code = generate_ccode(hyperbolic_regular, regularize)

hyperbolic_h = open("hyperbolic_regular.h", 'w')
hyperbolic_h.write(code)
hyperbolic_h.close()

print(code)

/*************************************
This code was generated automatically
using Sagemath and SymPy
**************************************/

// Subexpressions
double x0 = 2*shiftr_r;
double x1 = 2*lapse;
double x2 = krr*x1;
double x3 = 2*shiftz_z;
double x4 = kzz*x1;
double x5 = gzz*lapse;
double x6 = grr_z*x5;
double x7 = pow(gzz, 2);
double x8 = lapse*x7;
double x9 = 2*grz_r;
double x10 = gzz_z*lapse;
double x11 = krr*theta;
double x12 = grr*x7;
double x13 = grr_z*zz;
double x14 = grz_r*zz;
double x15 = (1.0/2.0)*grr;
double x16 = 1.0/grr;
double x17 = 1.0/x7;
double x18 = x16*x17;
double x19 = grr_z - grz_r;
double x20 = grr*x19;
double x21 = 2*krr;
double x22 = -grr_zz + grz_rz - gzz_rr + kzz*x21;
double x23 = grr*gzz*x1;
double x24 = pow(grr, 2);
double x25 = x1*zz;
double x26 = lapse*zz_z;
double x27 = gzz*x24;
double x28 = 1.0/x24;
double x29 = 1.0/gzz;
double x30 = x28*x29;
double x31 = x24*zz;
double x32 = kzz*lapse;
double x33 = grr*x5;
double x34 = pow(lapse, 2);

// Final